In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import Input, LocallyConnected1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 200
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 200
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 80, 16

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
# import os
# path='/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/sub_wise_process_TT/'
path='/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/'

In [15]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],200,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [16]:
n_length =  200
n_depth=8
n_channel=8
n_outputs=6

In [17]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,8)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model   

In [18]:
if __name__ == "__main__":
    model = generate_model()

(None, 200, 8)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
(None, 200, 8, 64)
<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>
(None, 102400)
(None, 6)
inputs_shape (None, 200, 8)


In [20]:
for i in range(1,37):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
#     feature standardization
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_length =  200
    n_depth=8
    n_channel=8
    x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5'
#     model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    test_mean=statistics.mean(test_acc)
    print('test_mean for %d subjects:',i)
    print(test_mean)

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 27000

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 9000

x_train shape:  (134, 200, 8)
134 training samples
y_train shape:  (134,)
num_time_periods 200
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (200, 8)
input_shape: (200, 8)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.

Epoch 1/120
9/9 [==============================] - 84s 3s/step - loss: 3.0784 - accuracy: 0.4347 - val_loss: 4.5539 - val_accuracy: 0.1818

Epoch 00001: val_accuracy improved from -inf to 0.18182, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5
Epoch 2/120
9/9 [==============================] - 1s 111ms/step - loss: 2.5847 - accuracy: 0.7295 - val_loss: 6.6372 - val_accuracy: 0.1818

Epoch 00002: val_accuracy did not improve from 0.18182
Epoch 3/120
9/9 [==============================] - 1s 106ms/step - loss: 2.1993 - accuracy: 0.8397 - val_loss: 4.8348 - val_accuracy: 0.1818

Epoch 00003: val_accuracy did not improve from 0.18182
Epoch 4/120
9/9 [==============================] - 2s 231ms/step - loss: 2.0051 - accuracy: 0.9330 - val_loss: 3.0404 - val_accuracy: 0.5000

Epoch 00004: val_accuracy improved from 0.18182 to 0.50000, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5
Epoch 5/120

9/9 [==============================] - 1s 110ms/step - loss: 0.7197 - accuracy: 0.9846 - val_loss: 1.4308 - val_accuracy: 0.7500

Epoch 00039: val_accuracy did not improve from 0.84091
Epoch 40/120
9/9 [==============================] - 1s 107ms/step - loss: 0.6910 - accuracy: 0.9985 - val_loss: 1.4556 - val_accuracy: 0.7500

Epoch 00040: val_accuracy did not improve from 0.84091
Epoch 41/120
9/9 [==============================] - 1s 105ms/step - loss: 0.7281 - accuracy: 0.9655 - val_loss: 1.4357 - val_accuracy: 0.7500

Epoch 00041: val_accuracy did not improve from 0.84091
Epoch 42/120
9/9 [==============================] - 1s 110ms/step - loss: 0.6944 - accuracy: 0.9883 - val_loss: 1.4362 - val_accuracy: 0.7500

Epoch 00042: val_accuracy did not improve from 0.84091
Epoch 43/120
9/9 [==============================] - 1s 115ms/step - loss: 0.6918 - accuracy: 0.9855 - val_loss: 1.4496 - val_accuracy: 0.7500

Epoch 00043: val_accuracy did not improve from 0.84091
Epoch 44/120
9/9 [=====

9/9 [==============================] - 1s 106ms/step - loss: 0.6028 - accuracy: 0.9970 - val_loss: 1.3624 - val_accuracy: 0.7955

Epoch 00080: val_accuracy did not improve from 0.84091
Epoch 81/120
9/9 [==============================] - 1s 107ms/step - loss: 0.5844 - accuracy: 1.0000 - val_loss: 1.3661 - val_accuracy: 0.7955

Epoch 00081: val_accuracy did not improve from 0.84091
Epoch 82/120
9/9 [==============================] - 1s 110ms/step - loss: 0.6107 - accuracy: 0.9860 - val_loss: 1.3565 - val_accuracy: 0.7955

Epoch 00082: val_accuracy did not improve from 0.84091
Epoch 83/120
9/9 [==============================] - 1s 106ms/step - loss: 0.5817 - accuracy: 1.0000 - val_loss: 1.3645 - val_accuracy: 0.7727

Epoch 00083: val_accuracy did not improve from 0.84091
Epoch 84/120
9/9 [==============================] - 1s 105ms/step - loss: 0.5878 - accuracy: 1.0000 - val_loss: 1.3326 - val_accuracy: 0.7955

Epoch 00084: val_accuracy did not improve from 0.84091
Epoch 85/120
9/9 [=====

3/3 [==============================] - 0s 36ms/step - loss: 1.7362 - accuracy: 0.8409
test_accuracy 0.8409090638160706
[0.8409090638160706]
test_mean for %d subjects: 1
0.8409090638160706
/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S2_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 27000

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/sub_wise_process_TT/S2_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 9000

x_train shape:  (134, 200, 8)
134 training samples
y_train shape:  (134,)
num_time_periods 200
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (200, 8)
input_shape: (200, 8)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]

...

9/9 [==============================] - 1s 115ms/step - loss: 0.2112 - accuracy: 1.0000 - val_loss: 0.6069 - val_accuracy: 0.8636

Epoch 00081: val_accuracy did not improve from 0.95455
Epoch 82/120
9/9 [==============================] - 1s 106ms/step - loss: 0.2112 - accuracy: 1.0000 - val_loss: 0.5521 - val_accuracy: 0.8864

Epoch 00082: val_accuracy did not improve from 0.95455
Epoch 83/120
9/9 [==============================] - 1s 110ms/step - loss: 0.2336 - accuracy: 0.9930 - val_loss: 0.5905 - val_accuracy: 0.8864

Epoch 00083: val_accuracy did not improve from 0.95455
Epoch 84/120
9/9 [==============================] - 1s 110ms/step - loss: 0.2104 - accuracy: 1.0000 - val_loss: 0.5769 - val_accuracy: 0.9091

Epoch 00084: val_accuracy did not improve from 0.95455
Epoch 85/120
9/9 [==============================] - 1s 108ms/step - loss: 0.2119 - accuracy: 1.0000 - val_loss: 0.6071 - val_accuracy: 0.9091

Epoch 00085: val_accuracy did not improve from 0.95455
Epoch 86/120
9/9 [=====

x_train shape:  (134, 200, 8)
134 training samples
y_train shape:  (134,)
num_time_periods 200
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (200, 8)
input_shape: (200, 8)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 

Epoch 1/120
9/9 [==============================] - 83s 2s/step - loss: 0.8447 - accuracy: 0.8787 - val_loss: 0.2989 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/CNN25X20/checkpoint.hdf5
Epoch 2/120
9/9 [==============================] - 1s 109ms/step - loss: 0.3336 - accuracy: 0.9755 - val_loss: 0.2992 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/120
9/9 [==============================] - 1s 107ms/step - loss: 0.4107 - accuracy: 0.9621 - val_loss: 0.2817 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/120
9/9 [==============================] - 1s 108ms/step - loss: 0.3887 - accuracy: 0.9676 - val_loss: 0.2728 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/120
9/9 [==============================] - 1s 114ms/step - loss: 0.3864 - accuracy: 0.9554 - val_loss: 0.2

Epoch 42/120
9/9 [==============================] - 1s 107ms/step - loss: 0.1337 - accuracy: 1.0000 - val_loss: 0.1314 - val_accuracy: 1.0000

Epoch 00042: val_accuracy did not improve from 1.00000
Epoch 43/120
9/9 [==============================] - 1s 109ms/step - loss: 0.1319 - accuracy: 1.0000 - val_loss: 0.1306 - val_accuracy: 1.0000

Epoch 00043: val_accuracy did not improve from 1.00000
Epoch 44/120
9/9 [==============================] - 1s 112ms/step - loss: 0.1348 - accuracy: 0.9945 - val_loss: 0.1301 - val_accuracy: 1.0000

Epoch 00044: val_accuracy did not improve from 1.00000
Epoch 45/120
9/9 [==============================] - 1s 105ms/step - loss: 0.1371 - accuracy: 0.9970 - val_loss: 0.1292 - val_accuracy: 1.0000

Epoch 00045: val_accuracy did not improve from 1.00000
Epoch 46/120
9/9 [==============================] - 1s 101ms/step - loss: 0.1354 - accuracy: 1.0000 - val_loss: 0.1295 - val_accuracy: 1.0000

Epoch 00046: val_accuracy did not improve from 1.00000
Epoch 47/1

9/9 [==============================] - 1s 105ms/step - loss: 0.1218 - accuracy: 1.0000 - val_loss: 0.1223 - val_accuracy: 1.0000

Epoch 00083: val_accuracy did not improve from 1.00000
Epoch 84/120
9/9 [==============================] - 1s 116ms/step - loss: 0.1223 - accuracy: 1.0000 - val_loss: 0.1224 - val_accuracy: 1.0000

Epoch 00084: val_accuracy did not improve from 1.00000
Epoch 85/120
9/9 [==============================] - 1s 112ms/step - loss: 0.1269 - accuracy: 0.9977 - val_loss: 0.1222 - val_accuracy: 1.0000

Epoch 00085: val_accuracy did not improve from 1.00000
Epoch 86/120
9/9 [==============================] - 1s 106ms/step - loss: 0.1316 - accuracy: 1.0000 - val_loss: 0.1219 - val_accuracy: 1.0000

Epoch 00086: val_accuracy did not improve from 1.00000
Epoch 87/120
9/9 [==============================] - 1s 109ms/step - loss: 0.1562 - accuracy: 0.9901 - val_loss: 0.1216 - val_accuracy: 1.0000

Epoch 00087: val_accuracy did not improve from 1.00000
Epoch 88/120
9/9 [=====

In [21]:
statistics.mean(test_acc)

0.9558080848720338